In [7]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt # https://matplotlib.org/stable/api/pyplot_summary.html#module-matplotlib.pyplot

%matplotlib inline

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats("retina") 
%config InlineBackend.figure_format='retina'

import warnings
warnings.filterwarnings('ignore')

print(f"{np.__version__}")
print(f"{pd.__version__}")
print(f"{mpl.__version__}")

1.23.5
1.5.3
3.7.1


In [9]:
import scipy as sp
import sympy
import sklearn

sympy.init_printing(use_latex='mathjax') # Juypter 노트북에서 수학식의 LaTeX 표현을 위해 필요함

print(f"{sklearn.__version__}")
print(f"{sympy.__version__}")
print(f"{sp.__version__}")

1.1.3
1.11.1
1.10.0


In [1]:
import os

base_src = "./data/"

# users
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zip_code"]
users = pd.read_csv(u_user_src, sep="|", names=u_cols, encoding="latin-1")
users = users.set_index("user_id")

# items
u_item_src = os.path.join(base_src, "u.item")
i_cols = [
    "movie_id", "title", "release_date", "video_release_date", "imdb_url", 
    "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-fi", "Thriller", "War", "Western"
    ]
movies = pd.read_csv(u_item_src, sep="|", names=i_cols, encoding="latin-1")
movies.set_index("movie_id")

# ratings
u_data_src = os.path.join(base_src, "u.data")
u_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_data_src, sep="\t", names=u_cols, encoding="latin-1")

users.shape, movies.shape, ratings.shape

NameError: name 'pd' is not defined

In [11]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

def score(model, neighbor_size=0):
    """
    이 섹션에서는 neighbor를 고려해야 하므로 여기서 모델에 주입해서 사용하기로 함.
    """
    id_pairs:tuple[int, int] = zip(x_test["user_id"], x_test["movie_id"])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test["rating"])
    return RMSE(y_true, y_pred)

## 사용자의 평가 경향을 고려한 CF 

1. 각 사용자의 평균 점수 계산
2. 편차 (평점 - 해당 사용자의 평균 점수) 사용
3. 편차 예측값 = 편차 * 다른 사용자 유사도
4. 예측값 = 편차 예측값 + 평점 평균

In [12]:
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings["rating"]

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.26, 
                                                    stratify=y)

ratings_matrix = x_train.pivot(
    index="user_id",
    columns="movie_id",
    values="rating"
)

ratings_matrix.head(1)

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,NaN,3.0,3.0,NaN,4.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy) # u-u sim
user_similarity = pd.DataFrame(data=user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

user_similarity.head(1)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.088978,0.052138,0.030152,0.218725,0.327315,0.308537,0.264006,0.050687,0.242015,...,0.230381,0.129382,0.202153,0.125288,0.151505,0.091778,0.266404,0.119847,0.146305,0.298911


In [43]:
# ratings_matrix |user_id, movie_id|
rating_mean = ratings_matrix.mean(axis=1)
# rating_mean |user_id, rating_mean|

# 각 점수에서 평균을 뺀다.
# |movie_id, user_id| - |user_id, rating_mean| -> |movie_id, user_id|
# rating_mean가 broadcast되어서 뺄셈이 수행된다.
rating_bias = (ratings_matrix.T - rating_mean).T

rating_bias

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,-0.607330,NaN,-0.60733,-0.60733,NaN,0.392670,NaN,NaN,-0.607330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.270833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.729167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.048780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.828571,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,0.517647,NaN,-0.482353,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias.columns:
        sim_score = user_similarity[user_id].copy() # 유사도 점수
        movie_ratings = rating_bias[movie_id].copy() # 영화 평점
        
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index # 평점이 없는 사람들의 인덱스
        movie_ratings = movie_ratings.dropna()
        sim_score = sim_score.drop(none_rating_idx)
        
        if neighbor_size == 0: # 0이면 일반 CF
            prediction = np.dot(sim_score, movie_ratings) / sim_score.sum()
            prediction = prediction + rating_mean[user_id] # 예측 + 편차
        
        else:
            if len(sim_score) > 1: # 일단 비슷한 유저 자체가 자기 자신 외에 더 있어야 하니까.
                # 지정한 이웃 숫자보다 유사도 점수 대상 유저가 적을 수도 있으니까. 10 이웃 목표했는데 5명 밖에 없는 경우.
                neighbor_size = min(neighbor_size, len(sim_score))
                
                sim_score = np.array(sim_score)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_score)
                
                sim_score = sim_score[user_idx][-neighbor_size:] # 오름차순으로 정렬 했으니까... 뒤에서부터 뽑아야 함.
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                
                prediction = np.dot(sim_score, movie_ratings) / sim_score.sum()
                prediction = prediction + rating_mean[user_id] # 예측 + 편차
            else:
                prediction = rating_mean[user_id]
    else: # ratings_matrix에 없는 영화는 3점으로 예측
        prediction = rating_mean[user_id]
        
    return prediction

In [47]:
# 와 최초로 1점 아랫대 나왔음 ㅋㅋ
score(CF_knn_bias, neighbor_size=30)

0.9476320043490462